In [1]:
import pickle
from pathlib import Path
from typing import Dict

import networkx as nx
import pandas as pd
from dowhy import CausalModel

from structure_data import choose_columns, preprocess_data
from sklearn.ensemble import HistGradientBoostingClassifier 

In [2]:
ROOT = Path('./')
RAW_DATA = ROOT / "raw_data/csv/eqls_2007and2011.csv"
DICT_PATH = ROOT / "data/dictionary.json"
GRAPH_PATH = ROOT / "graphs/full_causal.gpickle"
TREATMENT = "Y11_Q57"
OUTCOME = "Y11_MWIndex"


def load_data() -> pd.DataFrame:
    """Load and preprocess the raw EQLS data."""
    bdvs = ['Y11_EmploymentStatus', 'Y11_HHstructure', 'Y11_HHsize', 'Y11_Agecategory', 'Y11_Q7', 'Y11_Q31', 'Y11_Country', 'Y11_Q32', 'Y11_HH2a', TREATMENT, OUTCOME]
    df = choose_columns()
    df = preprocess_data(
        df,
        na_threshold=0.5,
        impute_strategy="drop",
        treatment_dichotomize_value="median",
        treatment_column=TREATMENT,
        backdoor_variables=bdvs
    )
    df.drop(
        columns=[c for c in df.columns if c not in bdvs]
    )
    return df


def load_graph() -> nx.DiGraph:
    """Load the causal graph describing relationships among variables."""
    with open(GRAPH_PATH, "rb") as f:
        return pickle.load(f)


def estimate_effects(df: pd.DataFrame, graph: nx.DiGraph) -> Dict[str, float]:
    """Estimate ATE using several DoWhy estimators."""
    model = CausalModel(
        data=df,
        treatment=TREATMENT,
        outcome=OUTCOME,
        graph=nx.nx_pydot.to_pydot(graph).to_string(),
    )
    estimand = model.identify_effect()
    methods = [
        "backdoor.propensity_score_matching",
        "backdoor.propensity_score_weighting",
        "backdoor.propensity_score_stratification",
        "backdoor.linear_regression",
        "backdoor.distance_matching",
    ]
    kwargs = {
        "backdoor.distance_matching": {
            "method_params": {'distance_metric': "minkowski", 'p': 2}
        }
    }
    results: Dict[str, float] = {}
    for m in methods:
        try:
            est = model.estimate_effect(estimand, method_name=m, **kwargs.get(m, {}))
            results[m] = float(est.value)
        except Exception as e:
            print(f"[!] Estimation with {m} failed: {e}")
            results[m] = float("nan")
    return results

In [3]:
df = load_data()   
graph = load_graph()
ps = HistGradientBoostingClassifier(random_state=42)
results = estimate_effects(df, graph)
print("Estimation results (ATE):")
for name, val in results.items():
    print(f"  {name:<40} {val:.4f}")

[+] Treatment column: Y11_Q57
[+] Outcome column: Y11_MWIndex
[+] Covariate columns: ['Y11_Country', 'Y11_Q31', 'Y11_Agecategory', 'Y11_HH2a', 'Y11_HHstructure', 'Y11_Q42', 'Y11_Q18', 'Y11_Q44', 'Y11_Q32', 'Y11_HHsize', 'Y11_Accommproblems', 'Y11_Q40a', 'Y11_Q40b', 'Y11_Q40c', 'Y11_Q40d', 'Y11_Q40e', 'Y11_Q40f', 'Y11_Q40g', 'Y11_Q7', 'Y11_EmploymentStatus', 'Y11_RuralUrban', 'Y11_Strainbasedconflict', 'Y11_Q15', 'Y11_Q16', 'Y11_Q12a', 'Y11_Q50d', 'Y11_Q53a']
[+] Y11_Country: 0 NA values (0.00%)
[+] Y11_Q31: 514 NA values (0.65%)
[+] Y11_Agecategory: 0 NA values (0.00%)
[+] Y11_HH2a: 0 NA values (0.00%)
[+] Y11_HHstructure: 0 NA values (0.00%)
[+] Y11_Q42: 141 NA values (0.18%)
[+] Y11_Q18: 324 NA values (0.41%)
[+] Y11_Q44: 56769 NA values (71.61%)
[+] Y11_Q32: 501 NA values (0.63%)
[+] Y11_HHsize: 0 NA values (0.00%)
[+] Y11_Accommproblems: 748 NA values (0.94%)
[+] Y11_Q40a: 1181 NA values (1.49%)
[+] Y11_Q40b: 43159 NA values (54.45%)
[+] Y11_Q40c: 630 NA values (0.79%)
[+] Y11_Q40d

/home/paperspace/EQLS-Causal-Project/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/paperspace/EQLS-Causal-Project/.venv/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = se

Estimation results (ATE):
  backdoor.propensity_score_matching       6.1956
  backdoor.propensity_score_weighting      7.0995
  backdoor.propensity_score_stratification 6.8991
  backdoor.linear_regression               6.8726
  backdoor.distance_matching               6.1869


In [6]:
df[['Y11_EmploymentStatus']].value_counts(normalize=True)

Y11_EmploymentStatus
1                       0.442068
4                       0.294929
5                       0.088433
2                       0.081762
6                       0.057135
3                       0.020519
7                       0.015154
Name: proportion, dtype: float64

In [5]:
df

,Y11_Q57,Y11_MWIndex,Y11_Country,Y11_Q31,Y11_Agecategory,Y11_HH2a,Y11_HHstructure,Y11_Q42,Y11_Q18,Y11_Q32,...,ips_normalized_weight,tips_normalized_weight,cips_normalized_weight,ips_stabilized_weight,tips_stabilized_weight,cips_stabilized_weight,d_y,dbar_y,strata,dbar
35635,0,64.0,15,4.0,2,1,1,1.0,3.0,0.0,...,0.000034,0.000040,0.000033,1.058225,0.317312,0.740913,0.0,64.0,383.0,1
35636,1,64.0,15,4.0,2,1,1,2.0,1.0,0.0,...,0.000064,0.000093,0.000054,0.687517,0.259087,0.428430,64.0,0.0,488.0,0
35637,1,80.0,15,3.0,5,1,1,1.0,1.0,0.0,...,0.000089,0.000093,0.000088,0.952991,0.259087,0.693904,80.0,0.0,314.0,0
35638,0,44.0,15,4.0,2,1,1,3.0,2.0,0.0,...,0.000035,0.000042,0.000033,1.076235,0.335322,0.740913,0.0,44.0,404.0,1
35639,0,80.0,15,4.0,3,1,1,1.0,1.0,0.0,...,0.000035,0.000041,0.000033,1.066296,0.325384,0.740913,0.0,80.0,393.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79264,1,36.0,28,1.0,4,2,2,2.0,5.0,2.0,...,0.000103,0.000093,0.000106,1.097756,0.259087,0.838668,36.0,0.0,209.0,0
79265,0,52.0,28,1.0,3,1,5,3.0,1.0,2.0,...,0.000038,0.000054,0.000033,1.176321,0.435408,0.740913,0.0,52.0,482.0,1
79267,1,52.0,28,1.0,4,1,2,2.0,1.0,2.0,...,0.000090,0.000093,0.000089,0.964079,0.259087,0.704992,52.0,0.0,306.0,0
79268,1,60.0,28,4.0,2,1,5,3.0,1.0,0.0,...,0.000085,0.000093,0.000082,0.905685,0.259087,0.646598,60.0,0.0,350.0,0
